In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough 
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [51]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=90,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

invoke_chain("My name is Potato")


content='Hello Potato! How can I assist you today?'


In [53]:
invoke_chain("What is my name?")

content='Your name is Potato. How can I assist you today, Potato?'
